In [ ]:
%%time
import win32com.client as client
from bs4 import BeautifulSoup
import unicodedata
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import pickle

from tensorflow.python.keras.backend import set_session
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import stringdist
import pandas as pd
import codecs
import re
import sys
import os

In [ ]:
from pdf2image import convert_from_path
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
tessdata_dir_config = r'--tessdata-dir "C:\Program Files\Tesseract-OCR\tessdata"'

In [ ]:
def w2v_tokenize_text(text):
    stop_words = stopwords.words('russian')
    stop_words_eng = stopwords.words('english')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на','from','FW','fw'])
    tokens = []
    tokens = word_tokenize(text)
    tokens = [i for i in tokens if (i not in string.punctuation)]
    tokens = [i for i in tokens if (i not in stop_words)]
    tokens = [i for i in tokens if (i not in stop_words_eng)]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    tokens = [i.replace("<", "").replace(">", "") for i in tokens]
    tokens = [i.lower() for i in tokens]
    tokens=' '.join(word for word in tokens)
    tokens=''.join([i for i in tokens if not i.isdigit()])
    return tokens

def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    stripped_text = unicodedata.normalize("NFKD", stripped_text)
    
    return stripped_text
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = ''
    for word in words:
        p = morph.parse(word)[0]
        
        res+= " "+ (p.normal_form)

    return res

In [ ]:
outlook = client.Dispatch('Outlook.Application')
namespace = outlook.GetNameSpace("MAPI")
account=namespace.Folders['info']
inbox = account.Folders['Входящие']
messages = inbox.Items
sess = tf.Session()
graph = tf.get_default_graph()
set_session(sess)
model = keras.models.load_model('MAILSCNN',compile=False)


encoder = LabelEncoder()
encoder.classes_ = np.load('classes2.npy', allow_pickle=True)
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
# messages = inbox.Items.restrict("[SentOn] > '24/12/2020 8:38 PM'")

In [ ]:
def check_if_pdf():
    ch=False
    for ii in messages[i].Attachments:
        if 'pdf' in str(ii):
            ch=True
    return ch

In [ ]:

def make_end_body(subjj,sendrr,sentonn):

    end_body=f"""</table> 
    <p></p>
    <div class=MsoNormal align=center style='text-align:center'>
           </div>
    <p class=MsoNormal>
        <b>
            Данное сообщение сформировано автоматически.
            <o:p></o:p>
        </b>
    </p>
    <p class=MsoNormal>
        <b>
            Если данное сообщение направлено не по назначению или его необходимо переслать дополнительным адресатам, просим уведомить нас, выбрав один из вариантов:
        </b>
    </p>
    <table class=MsoNormalTable border=0 cellspacing=0 cellpadding=0 style='border-collapse:collapse'>
      <tr style='height:3.75pt'>
          <td style='background:white;padding:0cm 0cm 0cm 0cm;height:3.75pt;background-color:#ffffff!'></td>
      </tr>
      <tr style='height:24.0pt'>
          <td width=30 style='width:22.0pt;background:#FF0000;padding:0cm 0cm 0cm 0cm;height:24.0pt;-webkit-border-radius: 0px;-moz-border-radius: 0px;border-radius: 5px'>
            <p class=MsoNormal align=center style='text-align:center'>
                <span style='color:white'>
                  <a href="mailto:concern@rosenergoatom.ru;bystrov-ai@rosenergoatom.ru?subject=Информирование об ошибочном адресовании_{subjj}&body=Письмо направлено не по назначению.%0DПрошу переслать письмо следующим адресатам: Укажите, если известно%0D%0D%0DFrom:[mailto:{sendrr}]%0DSent: {sentonn} %0DSubject: {subjj}"style="text-decoration: none"><b><span style='font-family:"Helvetica",sans-serif;color:black;text-decoration: none;'>Письмо направлено не по назначению.</span></b><b><span style='font-family:"Helvetica",sans-serif'> </span></b></a>
                  <o:p></o:p>
                </span>
            </p>
          </td>
      </tr>
      <tr style='height:3.75pt'>
          <td style='background:white;padding:0cm 0cm 0cm 0cm;height:3.75pt;background-color:#ffffff!important'></td>
      </tr>
      <tr style='height:24.0pt'>
          <td width=300 style='width:225.0pt;background:#FFD700;padding:0cm 0cm 0cm 0cm;height:24.0pt;-webkit-border-radius: 5px;-moz-border-radius: 5px;border-radius: 5px'>
            <p class=MsoNormal align=center style='text-align:center'>
                <span style='color:white'>
                  <a href="mailto:concern@rosenergoatom.ru;bystrov-ai@rosenergoatom.ru?subject=Информирование о дополнительных адресатах_{subjj}&body=Прошу направить письмо дополнительным адресатам: %0DУкажите, если известно%0D%0D%0DFrom:[mailto:{sendrr}]%0DSent: {sentonn} %0DSubject: {subjj}"style="text-decoration: none"><b><span style='font-family:"Helvetica",sans-serif;color:black'>Указаны не все адресаты</span></b><b><span style='font-family:"Helvetica",sans-serif'> </span></b></a>
                  <o:p></o:p>
                </span>
            </p>
          </td>
      </tr>
    </table>"""
    return end_body


In [ ]:
dicktt2={'sibileva@rosenergoatom.ru':'zheleznyakova-uv@rosenergoatom.ru',
        'skrynnikova-uv@rosenergoatom.ru':'skrynnikova-uv@rosenergoatom.ru;gubtullina-mr@rosenergoatom.ru',
        'chernikova-ov@rosenergoatom.ru':'',
        'karputova-nm@rosenergoatom.ru':'dpk@rosenergoatom.ru'}

# with open('replace_dict.pickle', 'rb') as handle:
#     dicktt2 = pickle.load(handle)
def checking(i):
    if i in dicktt2:
        print(i, 'SWAP')
        return dicktt2[i]
    else:
        print(i, 'NOSWAP')
        return i

In [ ]:
dicktt2

In [ ]:
checking('karputova-nm@rosenergoatom.ru')

In [ ]:
import datetime
import sqlite3
conn = sqlite3.connect('otchet.db')  # You can create a new database by changing the name within the quotes

def insert_one(conn, msg):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO zapis(subject,body_str,successfully,sender,reciever,spam,mistake,worktime)
              VALUES(?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    msg.append(datetime.datetime.now())
    cur.execute(sql, msg)
    conn.commit()
    return cur.lastrowid


In [ ]:
dickt_hard_matrix={'no-reply@rosreestr.ru':'shkadova-lu@rosenergoatom.ru',
                   'noreply-site@rosreestr.ru':'',
                   'naumova-ia@rosenergoatom.ru':'',
                   'chernikova-ov@rosenergoatom.ru':'',
                   'khmyrova-aa@rosenergoatom.ru':'',
                   'bystrov-ai@rosenergoatom.ru':'bystrov-ai@rosenergoatom.ru',
                  'klimov-au@rosenergoatom.ru':'',
                  'Dieva_E@nrcki.ru':'mihailova-m@rosenergoatom.ru;gustova@rosenergoatom.ru',
                  'nobody@hamailsender.fabrikant.ru':'duz@rosenergoatom.ru'}
def hard_matrix(msg):
    if msg.Class==43:
        if msg.SenderEmailType=='EX':
            mail=msg.Sender.GetExchangeUser().PrimarySmtpAddress
        else:
            mail=msg.SenderEmailAddress
    if mail in dickt_hard_matrix:
        resp=dickt_hard_matrix[mail]
    else:
        resp=False
    return resp

In [ ]:
def make_respond_KC(message):
     
    with open('counter.txt') as f:
        number = f.readlines()
#         ГГММДД
    regnumber='REA-%s/К%s'%(str(datetime.datetime.now().strftime('%Y%m%d'))[2:],number[0])
    htmlb="""
    <html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40"><head><meta http-equiv=Content-Type content="text/html; charset=utf-8"><meta name=Generator content="Microsoft Word 15 (filtered medium)">
    <style>
    @font-face
        {font-family:"Cambria Math";
        panose-1:2 4 5 3 5 4 6 3 2 4;}
    @font-face
        {font-family:Calibri;
        panose-1:2 15 5 2 2 2 4 3 2 4;}
    @font-face
        {font-family:"Rosatom Light";}
    /* Style Definitions */
    p.MsoNormal, li.MsoNormal, div.MsoNormal
        {margin:0cm;
        margin-bottom:.0001pt;
        font-size:11.0pt;
        font-family:"Calibri",sans-serif;}
    a:link, span.MsoHyperlink
        {mso-style-priority:99;
        color:#0563C1;
        text-decoration:underline;}
    a:visited, span.MsoHyperlinkFollowed
        {mso-style-priority:99;
        color:#954F72;
        text-decoration:underline;}
    p.MsoListParagraph, li.MsoListParagraph, div.MsoListParagraph
        {mso-style-priority:34;
        margin-top:0cm;
        margin-right:0cm;
        margin-bottom:0cm;
        margin-left:36.0pt;
        margin-bottom:.0001pt;
        font-size:11.0pt;
        font-family:"Calibri",sans-serif;}
    p.msonormal0, li.msonormal0, div.msonormal0
        {mso-style-name:msonormal;
        mso-margin-top-alt:auto;
        margin-right:0cm;
        mso-margin-bottom-alt:auto;
        margin-left:0cm;
        font-size:12.0pt;
        font-family:"Times New Roman",serif;}
    span.EmailStyle19
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle20
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle21
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle22
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle23
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle24
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle25
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle26
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle27
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle28
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle29
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle30
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle31
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle32
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle33
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle34
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle35
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle36
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle37
        {mso-style-type:personal-reply;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    .MsoChpDefault
        {mso-style-type:export-only;
        font-size:10.0pt;}
    @page WordSection1
        {size:612.0pt 792.0pt;
        margin:2.0cm 42.5pt 2.0cm 3.0cm;}
    div.WordSection1
        {page:WordSection1;}
    </style>
    </head>
    <body lang=RU link="#0563C1" vlink="#954F72">
        <div class=WordSection1>
           <p class=MsoNormal>
              <o:p>&nbsp;</o:p>
           </p>
           <p class=MsoNormal align=center style='text-align:center'>
              <img width=723 height=89 style='width:7.5333in;height:.925in' id="_x0000_i1025" src="cid:1.jpg">
              <o:p></o:p>
           </p>
           <div align=center>
              <table class=MsoNormalTable border=0 cellspacing=0 cellpadding=0 style='border-collapse:collapse'>
                 <tr style='height:46.2pt'>
                    <td width=563 style='width:541.5pt;border:none;border-bottom:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:46.2pt'>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <b>
                             <span style='font-size:16.0pt;font-family:"Rosatom Light";color:#222222'>
                                Уважаемый пользователь!
                                <o:p></o:p>
                             </span>
                          </b>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:16.0pt;font-family:"Rosatom Light";color:#222222'>Ваше обращение принято и направлено в работу</span>
                          <o:p></o:p>
                       </p>
                    </td>
                    <td width=3 style='width:2.25pt;padding:0cm 0cm 0cm 0cm;height:46.2pt'>
                       <p class=MsoNormal>
                          &nbsp;
                          <o:p></o:p>
                       </p>
                    </td>
                 </tr>
                 <tr style='height:252.15pt'>
                    <td width=563 style='width:541.5pt;padding:0cm 5.4pt 0cm 5.4pt;height:252.15pt'>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Номер обращения: 
                             <b>
                                %s
                                <o:p></o:p>
                             </b>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Тема обращения: &nbsp;
                             <b>
                                %s
                                <o:p></o:p>
                             </b>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Обращение принято: <b>%s</b>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Ожидаемый срок регистрации: <b>1 рабочий день.<span style='color:#1F497D'> </span></b>
                             <span style='color:#7030A0'>
                                <o:p></o:p>
                             </span>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>Для получения информаци<span style='color:black'>и</span> о регистрационном номере необходимо направить запрос в Контактный центр:</span>
                          <span style='color:#1F497D'>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <a href="mailto:info@rosenergoatom.ru?subject=%s"><span style='color:windowtext;text-decoration:none'><img border=0 width=280 height=40 style='width:2.9166in;height:.4166in' id="_x0000_i1026" src="cid:2.jpg"></span></a>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             При нажатии формируется письмо по
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             шаблону запроса с номером обращения
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light";color:#1F497D'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light";color:black'>В случае </span>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             возникновении вопросов просим обращаться в Контактный центр, указав номер обращения.
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Время работы Контактного центра:
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Пн - Чт &nbsp;с <b>09:00 по 18:00 (MSK)</b>.<span style='color:#1F497D'> </span>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Пт с <b>09:00 по 16:45 (MSK).</b>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span><img width=724 height=30 style='width:7.5416in;height:.3166in' src="cid:3.jpg" v:shapes="Рисунок_x0020_2"></span>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p></o:p>
                          </span>
                       </p>
                    </td>
                 </tr>
              </table>
           </div>
           <p class=MsoNormal>
              <span style='color:#1F497D'>
                 <o:p>&nbsp;</o:p>
              </span>
           </p>
        </div>
     </body>
     </html>"""%(regnumber,message.Subject,str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M %p")),regnumber)
    
    message.SaveAs(r"C:\Users\a-bystrov-ai\messattach.msg")
    time.sleep(5)
    NewMsg = outlook.CreateItem(0)
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\messattach.msg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\1.jpg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\2.jpg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\3.jpg")
    NewMsg.HTMLBody = htmlb
    NewMsg.Subject = regnumber+': '+message.Subject
    print(message.Sender.Address)
    NewMsg.To = message.Sender.Address+';concern@rosenergoatom.ru'
    NewMsg.Send()

    print(regnumber)



    towrite=int(number[0])+1
    with open('counter.txt','w') as f:
        f.write(str(towrite))

In [ ]:
def make_respond_II(message,reciever):
     
    with open('counter.txt') as f:
        number = f.readlines()
#         ГГММДД
    regnumber='REA-%s/И%s'%(str(datetime.datetime.now().strftime('%Y%m%d'))[2:],number[0])
    htmlb="""
    <html xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:w="urn:schemas-microsoft-com:office:word" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns="http://www.w3.org/TR/REC-html40"><head><meta http-equiv=Content-Type content="text/html; charset=utf-8"><meta name=Generator content="Microsoft Word 15 (filtered medium)">
    <style>
    @font-face
        {font-family:"Cambria Math";
        panose-1:2 4 5 3 5 4 6 3 2 4;}
    @font-face
        {font-family:Calibri;
        panose-1:2 15 5 2 2 2 4 3 2 4;}
    @font-face
        {font-family:"Rosatom Light";}
    /* Style Definitions */
    p.MsoNormal, li.MsoNormal, div.MsoNormal
        {margin:0cm;
        margin-bottom:.0001pt;
        font-size:11.0pt;
        font-family:"Calibri",sans-serif;}
    a:link, span.MsoHyperlink
        {mso-style-priority:99;
        color:#0563C1;
        text-decoration:underline;}
    a:visited, span.MsoHyperlinkFollowed
        {mso-style-priority:99;
        color:#954F72;
        text-decoration:underline;}
    p.MsoListParagraph, li.MsoListParagraph, div.MsoListParagraph
        {mso-style-priority:34;
        margin-top:0cm;
        margin-right:0cm;
        margin-bottom:0cm;
        margin-left:36.0pt;
        margin-bottom:.0001pt;
        font-size:11.0pt;
        font-family:"Calibri",sans-serif;}
    p.msonormal0, li.msonormal0, div.msonormal0
        {mso-style-name:msonormal;
        mso-margin-top-alt:auto;
        margin-right:0cm;
        mso-margin-bottom-alt:auto;
        margin-left:0cm;
        font-size:12.0pt;
        font-family:"Times New Roman",serif;}
    span.EmailStyle19
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle20
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle21
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle22
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle23
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle24
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle25
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle26
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle27
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle28
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle29
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle30
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:windowtext;}
    span.EmailStyle31
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle32
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle33
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle34
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle35
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle36
        {mso-style-type:personal;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    span.EmailStyle37
        {mso-style-type:personal-reply;
        font-family:"Calibri",sans-serif;
        color:#1F497D;}
    .MsoChpDefault
        {mso-style-type:export-only;
        font-size:10.0pt;}
    @page WordSection1
        {size:612.0pt 792.0pt;
        margin:2.0cm 42.5pt 2.0cm 3.0cm;}
    div.WordSection1
        {page:WordSection1;}
    </style>
    </head>
    <body lang=RU link="#0563C1" vlink="#954F72">
        <div class=WordSection1>
           <p class=MsoNormal>
              <o:p>&nbsp;</o:p>
           </p>
           <p class=MsoNormal align=center style='text-align:center'>
              <img width=723 height=89 style='width:7.5333in;height:.925in' id="_x0000_i1025" src="cid:1.jpg">
              <o:p></o:p>
           </p>
           <div align=center>
              <table class=MsoNormalTable border=0 cellspacing=0 cellpadding=0 style='border-collapse:collapse'>
                 <tr style='height:46.2pt'>
                    <td width=563 style='width:541.5pt;border:none;border-bottom:solid windowtext 1.0pt;padding:0cm 5.4pt 0cm 5.4pt;height:46.2pt'>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <b>
                             <span style='font-size:16.0pt;font-family:"Rosatom Light";color:#222222'>
                                Уважаемый пользователь!
                                <o:p></o:p>
                             </span>
                          </b>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:16.0pt;font-family:"Rosatom Light";color:#222222'>Ваше обращение принято и направлено в работу</span>
                          <o:p></o:p>
                       </p>
                    </td>
                    <td width=3 style='width:2.25pt;padding:0cm 0cm 0cm 0cm;height:46.2pt'>
                       <p class=MsoNormal>
                          &nbsp;
                          <o:p></o:p>
                       </p>
                    </td>
                 </tr>
                 <tr style='height:252.15pt'>
                    <td width=563 style='width:541.5pt;padding:0cm 5.4pt 0cm 5.4pt;height:252.15pt'>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Номер обращения: 
                             <b>
                                %s
                                <o:p></o:p>
                             </b>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Тема обращения: &nbsp;
                             <b>
                                %s
                                <o:p></o:p>
                             </b>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Обращение принято: <b>%s</b>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Ожидаемый срок регистрации: <b>1 рабочий день.<span style='color:#1F497D'> </span></b>
                             <span style='color:#7030A0'>
                                <o:p></o:p>
                             </span>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>Для получения информаци<span style='color:black'>и</span> о регистрационном номере необходимо направить запрос в Контактный центр:</span>
                          <span style='color:#1F497D'>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <a href="mailto:info@rosenergoatom.ru?subject=%s"><span style='color:windowtext;text-decoration:none'><img border=0 width=280 height=40 style='width:2.9166in;height:.4166in' id="_x0000_i1026" src="cid:2.jpg"></span></a>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             При нажатии формируется письмо по
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             шаблону запроса с номером обращения
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light";color:#1F497D'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light";color:black'>В случае </span>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             возникновении вопросов просим обращаться в Контактный центр, указав номер обращения.
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Время работы Контактного центра:
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Пн - Чт &nbsp;с <b>09:00 по 18:00 (MSK)</b>.<span style='color:#1F497D'> </span>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             Пт с <b>09:00 по 16:45 (MSK).</b>
                             <o:p></o:p>
                          </span>
                       </p>
                       <p class=MsoNormal>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p>&nbsp;</o:p>
                          </span>
                       </p>
                       <p class=MsoNormal align=center style='text-align:center'>
                          <span><img width=724 height=30 style='width:7.5416in;height:.3166in' src="cid:3.jpg" v:shapes="Рисунок_x0020_2"></span>
                          <span style='font-size:12.0pt;font-family:"Rosatom Light"'>
                             <o:p></o:p>
                          </span>
                       </p>
                    </td>
                 </tr>
              </table>
           </div>
           <p class=MsoNormal>
              <span style='color:#1F497D'>
                 <o:p>&nbsp;</o:p>
              </span>
           </p>
        </div>
     </body>
     </html>"""%(regnumber,message.Subject,str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M %p")),regnumber)
    message.SaveAs(r"C:\Users\a-bystrov-ai\messattach.msg")
    time.sleep(5)
    NewMsg = outlook.CreateItem(0)
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\messattach.msg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\1.jpg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\2.jpg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\3.jpg")
    NewMsg.HTMLBody = htmlb
    NewMsg.Subject = regnumber+": "+message.Subject
    print(message.Sender.Address)
    NewMsg.To = message.Sender.Address
    NewMsg.Send()
    time.sleep(1)
    NewMsg = outlook.CreateItem(0)
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\messattach.msg")
    time.sleep(5)
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\1.jpg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\2.jpg")
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\3.jpg")
    NewMsg.HTMLBody = 'Перенаправлено на %s<p></p>'%reciever+htmlb
    NewMsg.Subject = regnumber+": "+message.Subject
    print(message.Sender.Address)
    NewMsg.To = 'concern@rosenergoatom.ru'
    NewMsg.Send()

    print(regnumber)



    towrite=int(number[0])+1
    with open('counter.txt','w') as f:
        f.write(str(towrite))

In [ ]:
def concern_send(message):
    make_respond_KC(message)

In [ ]:
def spam_check_concern(message):
    print('SPAM_SEND_CONCERN')
    message.SaveAs(r"C:\Users\a-bystrov-ai\messattach.msg")
    time.sleep(3)
    NewMsg = outlook.CreateItem(0)
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\messattach.msg")
    NewMsg.Subject = "[СПАМ]: "+message.Subject
    NewMsg.To = 'concern@rosenergoatom.ru'
    NewMsg.Send()

In [ ]:
def predict_nopdf(outlook):
    print('NOPDF')
    text=strip_html_tags(messages[i].body)
    text=messages[i].Subject+' '+text
    print('text_get', strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    text=w2v_tokenize_text(text)
    text=lemmatize(text)
    text=re.sub(r'^.*?subject ', 'subject ', text)
    one_pred=tokenizer.texts_to_sequences(texts=[text])
    one_padded=pad_sequences(one_pred, maxlen=140,truncating='post')
    with graph.as_default():
        set_session(sess)
        prob=model.predict(one_padded)

        item=prob.argmax()
        isok=prob[0][item]
        reciever=checking(encoder.inverse_transform(np.argmax(prob, axis=-1))[0])
        RESP=hard_matrix(messages[i])
        if RESP!=False:
            print('HARDMATRIX')
            reciever=checking(RESP)
            isok=0.99
        try:
            if (isok>0.85) and (messages[i].Sender.Address!='/O=rosenergoatom/OU=ROSENERGOATOM/cn=Recipients/cn=rpa-support13d') and reciever!='NoReply' and 'часть' not in messages[i].Subject.lower() and 'часть' not in messages[i].body.lower():
                print('0.95',"|",isok)
                if (encoder.inverse_transform(np.argmax(prob, axis=-1))=='спам')  :
                    if messages[i].Sender.Address.lower() in list_prove:
                        pass
                    else:
                        if isok>0.95:
                            messages[i].GetInspector()
                            messages[i].Categories='Робот(СПАМ)'
                            messages[i].Save()
                            time.sleep(5)
                            spam_check_concern(messages[i])
                            print('SPAM')
                            insert_one(conn,['spam',messages[i].HTMLBody,1,messages[i].Sender.Address,'None',1,0])
                        else:
                            print('Ne perenapravleno ', isok, messages[i].Subject,strftime("%Y-%m-%d %H:%M:%S", gmtime()),reciever)
                            messages[i].GetInspector()
                            messages[i].Categories='Робот(не перенаправлено)'
                            messages[i].Save()
                            insert_one(conn,[messages[i].Subject,messages[i].HTMLBody,0,messages[i].Sender.Address,reciever,0,0])
                            time.sleep(2)
                            make_respond_KC(messages[i])
                    time.sleep(3)
    #                     print(messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p"),'|',encoder.inverse_transform(np.argmax(prob, axis=-1)),'|',isok,'|',text)
                else:
                    dicttrue=False
                    if reciever in dicktt:
                        print(1)
                        if isok>dicktt[reciever] and exceptions(reciever):
                            print(5)
                            dicttrue=True
                        else:
                            print(4)
                            dicttrue=False
                    elif (reciever not in dicktt) and (isok>0.95) and exceptions(reciever):
                        print(2)
                        dicttrue=True
                    print(3)
                    if dicttrue==True:
                        print('to_check', messages[i].Subject)
                        for att in messages[i].Attachments:
                            att.SaveASFile(os.path.join(get_path,att.FileName))
                        print('get att')
                        msg=outlook.CreateItem(0)
                        text_forward="""
                        <div style='border:none;border-top:solid #B5C4DF 1.0pt;padding:3.0pt 0cm 0cm 0cm'>
                         <p class=MsoNormal>
                            <b><span style='font-size:10.0pt;font-family:"Tahoma","sans-serif"'>From:</span></b>
                            <span style='font-size:10.0pt;font-family:"Tahoma","sans-serif"'>
                               [mailto:%s] <br><b>Sent:</b> %s<br><b>To:</b> %s<br><b>Subject:</b> %s
                               <o:p></o:p>
                            </span>
                         </p>
                        </div>
                        """%(messages[i].Sender.Address,str(messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p")),messages[i].To,messages[i].Subject)
                        msg.HTMLBody = text_forward+messages[i].HTMLBody+'<p></p>'+make_end_body(messages[i].Subject,messages[i].Sender.Address,messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p"))
                        msg.To=reciever
                        msg.Subject=messages[i].Subject
        #                 print(messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p"),'|',reciever,'|',isok,'|',text)
                        for ii in glob.glob(r"C:\Users\a-bystrov-ai\Desktop\attach\*"):
        #                     print(ii)
                            msg.Attachments.Add(ii)
    # 
                        msg.Send()
                        print('SENTTORECIEVER')
                        print(reciever)
                        time.sleep(1)
                        messages[i].GetInspector()
                        messages[i].Categories='Робот(Входящее)'
                        messages[i].Save()
                        time.sleep(1)
                        pythoncom.CoInitialize()
                        outlook = client.Dispatch('Outlook.Application')
                        namespace = outlook.GetNameSpace("MAPI")
                        account=namespace.Folders['rpa-support@rosenergoatom.ru']
                        NewMsg = outlook.CreateItem(0)
                        NewMsg.HTMLBody = messages[i].Subject+'<p></p>'+messages[i].HTMLBody
                        NewMsg.Subject = 'Перенаправлено %s'%reciever
                        NewMsg.To = "info@rosenergoatom.ru"
                        NewMsg.Send()
                        print('SENTinfo')

                        for f in glob.glob(r"C:\Users\a-bystrov-ai\Desktop\attach\*"):
                            os.remove(f)



                        print('PERENAPRAVLENO ',strftime("%Y-%m-%d %H:%M:%S", gmtime()))
                        insert_one(conn,[messages[i].Subject,messages[i].HTMLBody,1,messages[i].Sender.Address,reciever,0,0])
                        time.sleep(1)
                        time.sleep(1)
                        make_respond_II(messages[i],reciever)
                    else:
                        print('Ne perenapravleno ', isok, messages[i].Subject,strftime("%Y-%m-%d %H:%M:%S", gmtime()),reciever)
                        messages[i].GetInspector()
                        messages[i].Categories='Робот(не перенаправлено)'
                        messages[i].Save()
                        insert_one(conn,[messages[i].Subject,messages[i].HTMLBody,0,messages[i].Sender.Address,reciever,0,0])
                        time.sleep(1)
                        make_respond_KC(messages[i])
                        

            else:
                print('Ne perenapravleno ', isok, messages[i].Subject,strftime("%Y-%m-%d %H:%M:%S", gmtime()),reciever)
                messages[i].GetInspector()
                messages[i].Categories='Робот(не перенаправлено)'
                messages[i].Save()
                insert_one(conn,[messages[i].Subject,messages[i].HTMLBody,0,messages[i].Sender.Address,reciever,0,0])
                time.sleep(1)
                make_respond_KC(messages[i])

        except Exception as e:
            print(e)
            messages[i].GetInspector()
            messages[i].Categories='Робот(не перенаправлено)'
            messages[i].Save()
            print('???')
            insert_one(conn,[messages[i].Subject,'error',0,messages[i].Sender.Address,reciever,0,0])
            time.sleep(1)
            make_respond_KC(messages[i])

In [ ]:
from PyPDF2 import PdfFileReader, PdfFileWriter
path=r"C:\Users\a-bystrov-ai\pdfs_prod"
import fitz
import glob
with open(r"vectorizer.pickle", 'rb') as handle:
    vectorizer = pickle.load(handle)
with open(r"logit.pickle", 'rb') as handle:
    logit = pickle.load(handle)
with open(r"rf.pickle", 'rb') as handle:
    rf = pickle.load(handle)
enc_final = LabelEncoder()
enc_final.classes_ = np.load(r"classes.npy", allow_pickle=True)
def one_page(file):
    
    try:
        first_page = convert_from_path(file )[0] 
        tt = str(((pytesseract.image_to_string(first_page, lang= 'rus', config=tessdata_dir_config))))
        if 'по списку рассылки' in tt.lower():
            second_page = convert_from_path(file)[1] 
            tt += str(((pytesseract.image_to_string(second_page, lang= 'rus', config=tessdata_dir_config))))
        
        os.remove(file)
    except:
        del(first_page)
        del(tt)
        del(second_page)
        print("failed to extract text from pdf")
        tt=''
    return tt
# def extracttext():
#     root=r"C:\\Users\\rpa-support\\pdfs_prod"
#     asd=''
#     text=''
#     pdftext=''
#     for path, subdirs, files in os.walk(root):
        
#         for ii in files:
#             i=root+"\\\\"+ii
#             filename, file_extension = os.path.splitext(i)
#             try:
#                 if file_extension == '.pdf':

#                     doc = fitz.open(i)
#                     for page in doc:
#             #                 pfdtext += page.getText()
#                         pdftext+=doc[0].getText()
#                     doc.close()
#                     if pdftext =='':

#                         text=textract.process(i,method='tesseract',language='rus').decode('utf-8')
                        

#                     else:
#                         text = textract.process(i,language='rus').decode('utf-8')
                        
    
#             except Exception as e:
#                 print(e)
#             asd=asd+text+';m;m;m'
#     files = glob.glob(path + '\\*' )
#     for f in files:
#         os.remove(f)
    
#     return asd

In [ ]:
def get_text_pdf(mes):
    join_text = ''
    for attach in mes.attachments:
        try:
            attachment_name = str(attach).lower()
            filename, file_extension = os.path.splitext(attachment_name)
            if file_extension =='.pdf':
                attach.SaveASFile(path + '\\' + attachment_name)
                t = one_page(path + '\\' + attachment_name)
                join_text += t+';m;m;m'
        except Exception as e:
            print(e)
            continue
    return join_text

In [ ]:
def predictions(mes):
    text = get_text_pdf(mes)
    #print(text)
    text= ''.join([i for i in text if not i.isdigit()])
    text = w2v_tokenize_text(text)
    text=vectorizer.transform([text])
    y_pred_proba=logit.predict_proba(text)
    df_logit = pd.DataFrame(y_pred_proba)
    text=strip_html_tags(mes.body)
    text=mes.Subject+' '+text
    text=w2v_tokenize_text(text)
    text=lemmatize(text)
    text=re.sub(r'^.*?subject ', 'subject ', text)
    one_pred=tokenizer.texts_to_sequences(texts=[text])
    nn_df = pd.DataFrame()
    one_padded=pad_sequences(one_pred, maxlen=140,truncating='post')
    with graph.as_default():
        prob=model.predict(one_padded)
    nn_df = nn_df.append(pd.DataFrame(prob),ignore_index= True )
    df_logit.reset_index(drop=True,inplace=True)
    nn_df.reset_index(drop=True, inplace =True)
    merge_df = df_logit.merge(nn_df, left_on = df_logit.index, right_on = nn_df.index, suffixes=('_logit', '_nn'))
    merge_df.drop('key_0', axis=1, inplace=True)
    y_pred = rf.predict_proba(merge_df)
    return y_pred

In [ ]:
def check_matrix(mes, outlook):
    text = get_text_pdf(mes)
    text = text.lower()
    em = ''
    fltr =''
    isp_ob = len(re.findall(r'и\.\s*о\.', text))
    print(isp_ob)
    if 'исполняющему обязанности' not in text or isp_ob == 0:
        print('I.O.')
        for row in range(mtrx.shape[0]):
            regex = mtrx.iloc[row,0]

            regex1= mtrx.iloc[row, 3]
            io = mtrx.iloc[row,2]
            reg = "уважаем(ая|ый) {}".format(io)
            if len(re.findall(regex, text))!=0 or len(re.findall(regex1, text))!=0:
                if mtrx.iloc[row,1] not in em:
                    print(mtrx.iloc[row,1])
                    em_checked = checking(mtrx.iloc[row,1])
                    em += em_checked+ ";"
        if re.findall(';', em) == 1 and len(re.findall(reg, text))==0:
            em = ''

        if em != "":
            print("--------------------------------WORKING WITH THE MATRIX--------------------------------")
            print(em)
            reciever = em
            print(reciever)
            for att in mes.Attachments:
                att.SaveASFile(os.path.join(get_path,att.FileName))
            print('get att')
            msg=outlook.CreateItem(0)
            text_forward="""
            <div style='border:none;border-top:solid #B5C4DF 1.0pt;padding:3.0pt 0cm 0cm 0cm'>
             <p class=MsoNormal>
                <b><span style='font-size:10.0pt;font-family:"Tahoma","sans-serif"'>From:</span></b>
                <span style='font-size:10.0pt;font-family:"Tahoma","sans-serif"'>
                   [mailto:%s] <br><b>Sent:</b> %s<br><b>To:</b> %s<br><b>Subject:</b> %s
                   <o:p></o:p>
                </span>
             </p>
            </div>
            """%(mes.Sender.Address,str(mes.SentOn.strftime("%m/%d/%Y %H:%M %p")),mes.To,mes.Subject)
            msg.HTMLBody = text_forward+mes.HTMLBody+'<p></p>'+make_end_body(messages[i].Subject,messages[i].Sender.Address,messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p"))
            msg.Subject=mes.Subject
            msg.To=reciever
    #                 print(messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p"),'|',reciever,'|',isok,'|',text)
            for ii in glob.glob(r"C:\Users\a-bystrov-ai\Desktop\attach\*"):
    #                     print(ii)
                msg.Attachments.Add(ii)

            msg.Send()
            print('SENTTORECIEVER')
            print(reciever)
            time.sleep(1)
            mes.GetInspector()
            mes.Categories='Робот(Входящее)'
            mes.Save()
            time.sleep(1)
            pythoncom.CoInitialize()
            outlook = client.Dispatch('Outlook.Application')
            namespace = outlook.GetNameSpace("MAPI")
            account=namespace.Folders['rpa-support@rosenergoatom.ru']
            NewMsg = outlook.CreateItem(0)
            NewMsg.HTMLBody = mes.Subject+'<p></p>'+mes.HTMLBody
            NewMsg.Subject = 'Перенаправлено %s'%reciever
            NewMsg.To = "info@rosenergoatom.ru"
            NewMsg.Send()
            
            print('SENTinfo')
            for f in glob.glob(r"C:\Users\a-bystrov-ai\Desktop\attach\*"):
                os.remove(f)
            print('PERENAPRAVLENO ',strftime("%Y-%m-%d %H:%M:%S", gmtime()))
            insert_one(conn,[mes.Subject,mes.HTMLBody,1,mes.Sender.Address,reciever,0,0])
            time.sleep(1)
            time.sleep(1)
            make_respond_II(messages[i],reciever)
            fltr = 'stop'
        else:
            fltr = 'continue'
    else:
        fltr = 'continue'
    return fltr

In [ ]:
dicktt={'dpk@rosenergoatom.ru':0.95,
        'zheleznyakova-uv@rosenergoatom.ru':0.9,
     'duz@rosenergoatom.ru':0.9,
     'karputova-nm@rosenergoatom.ru':0.99,
     'karputova-nm@rosenergoatom.ru;dpk@rosenergoatom.ru':0.99,
     'naumova-ia@rosenergoatom.ru':0.999,
     'tenison-te@rosenergoatom.ru':0.9,
     'klevoshina@rosenergoatom.ru':0.9,
    'beloshapkina-mm@rosenergoatom.ru':0.9}

In [ ]:
def exceptions(reciever):
    listt=[]
    if messages[i].Sender.Address!='/O=rosenergoatom/OU=ROSENERGOATOM/cn=Recipients/cn=rpa-support13d':
        listt.append(True)
    else:
        listt.append(False)
    if reciever!='NoReply':
        listt.append(True)
    else:
        listt.append(False)
    if 'часть' not in messages[i].Subject.lower():
        listt.append(True)
    else:
        listt.append(False)
#     if 'часть' not in messages[i].body.lower():
#         listt.append(True)
#     else:
#         listt.append(False)
    if False in listt:
        return False
    else:
        return True

In [ ]:
def predict_pdf(outlook):
    print("PDF")
    pred=predictions(messages[i])
    print(pred)
    reciever=checking(enc_final.inverse_transform(np.argmax(pred, axis=-1))[0])
    print(reciever)
    isok=pred.max()
    dicttrue=False
    if reciever in dicktt:
        print(1)
        if isok>dicktt[reciever] and exceptions(reciever):
            print(5)
            dicttrue=True
        else:
            print(4)
            dicttrue=False
    elif (reciever not in dicktt) and (isok>0.95) and exceptions(reciever):
        print(2)
        dicttrue=True
    print(3)
    if dicttrue==True:
        print('0.95',"|",isok)

        print('to_check', messages[i].Subject)
        for att in messages[i].Attachments:
            att.SaveASFile(os.path.join(get_path,att.FileName))
        print('get att')
        msg=outlook.CreateItem(0)
        text_forward="""
        <div style='border:none;border-top:solid #B5C4DF 1.0pt;padding:3.0pt 0cm 0cm 0cm'>
         <p class=MsoNormal>
            <b><span style='font-size:10.0pt;font-family:"Tahoma","sans-serif"'>From:</span></b>
            <span style='font-size:10.0pt;font-family:"Tahoma","sans-serif"'>
               [mailto:%s] <br><b>Sent:</b> %s<br><b>To:</b> %s<br><b>Subject:</b> %s
               <o:p></o:p>
            </span>
         </p>
        </div>
        """%(messages[i].Sender.Address,str(messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p")),messages[i].To,messages[i].Subject)
        msg.HTMLBody = text_forward+messages[i].HTMLBody+'<p></p>'+make_end_body(messages[i].Subject,messages[i].Sender.Address,messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p"))
        msg.Subject=messages[i].Subject
        msg.To=reciever
#                 print(messages[i].SentOn.strftime("%m/%d/%Y %H:%M %p"),'|',reciever,'|',isok,'|',text)
        for ii in glob.glob(r"C:\Users\a-bystrov-ai\Desktop\attach\*"):
#                     print(ii)
            msg.Attachments.Add(ii)

        msg.Send()
        print('SENTTORECIEVER')
        print(reciever)
        time.sleep(1)
        messages[i].GetInspector()
        messages[i].Categories='Робот(Входящее)'
        messages[i].Save()
        time.sleep(1)
        pythoncom.CoInitialize()
        outlook = client.Dispatch('Outlook.Application')
        namespace = outlook.GetNameSpace("MAPI")
        account=namespace.Folders['rpa-support@rosenergoatom.ru']
        NewMsg = outlook.CreateItem(0)
        NewMsg.HTMLBody = messages[i].Subject+'<p></p>'+messages[i].HTMLBody
        NewMsg.Subject = 'Перенаправлено %s'%reciever
        NewMsg.To = "info@rosenergoatom.ru"
        NewMsg.Send()
        print('SENTinfo')
        for f in glob.glob(r"C:\Users\a-bystrov-ai\Desktop\attach\*"):
            os.remove(f)
        print('PERENAPRAVLENO ',strftime("%Y-%m-%d %H:%M:%S", gmtime()))
        insert_one(conn,[messages[i].Subject,messages[i].HTMLBody,1,messages[i].Sender.Address,reciever,0,0])
        time.sleep(1)
        time.sleep(1)
        make_respond_II(messages[i],reciever)
    else:
        print('Ne perenapravleno ', isok, messages[i].Subject,strftime("%Y-%m-%d %H:%M:%S", gmtime()),reciever)
        messages[i].GetInspector()
        messages[i].Categories='Робот(не перенаправлено)'
        messages[i].Save()
        insert_one(conn,[messages[i].Subject,messages[i].HTMLBody,0,messages[i].Sender.Address,reciever,0,0])
        time.sleep(1)
        make_respond_KC(messages[i])

In [ ]:
def one_page_KT(file):
    
#     try:
    first_page = convert_from_path(file)[0]
    tt = str(pytesseract.image_to_string(first_page, lang= 'eng', config=tessdata_dir_config))
    ttt = str(pytesseract.image_to_string(first_page, lang= 'rus', config=tessdata_dir_config))
    os.remove(file)
    tt+=ttt
#     except Exception as e:
#         print(e)
    return tt

def get_text_pdf_KT(mes):
    join_text = ''
    for attach in mes.attachments:
        try:
            attachment_name = str(attach).lower()
            filename, file_extension = os.path.splitext(attachment_name)
            if file_extension =='.pdf':
                attach.SaveASFile(path + '\\' + attachment_name)
                t = one_page_KT(path + '\\' + attachment_name)
                join_text += t+';m;m;m'
        except Exception as e:
            print(e)
            continue
    return join_text

def ktdsp(mes,outlook):
    dsp=False
    text = get_text_pdf_KT(mes)
    print(len(text),' длинна текста')
    keys = ['ДСП','non-public','Non-Public','Non-public']
    for i in keys:
        if i in text:
            dsp=True
            break
    if dsp == True:
        print('Ne perenapravleno ', isok, mes.Subject,strftime("%Y-%m-%d %H:%M:%S", gmtime()),reciever)
        mes.GetInspector()
        mes.Categories='Робот(не перенаправлено)'
        mes.Save()
        insert_one(conn,[mes.Subject,mes.HTMLBody,0,mes.Sender.Address,reciever,0,0])
        time.sleep(1)
        make_respond_KC(mes)
        return True
    else:
        return False

In [ ]:
with open ('itemlist.pkl', 'rb') as fp:
    itemlist = pickle.load(fp)
itemlist.append('a.shinkarenko@zzt.ru')
ext=['pdf','zip','rar','7z','docx']
def check_delay(mes):
    listt=[]
    extcheck=False
    if mes.SenderEmailAddress in itemlist:
        listt.append(True)
    else:
        listt.append(False)
    if mes.Attachments:
        for f in mes.Attachments:
            exten=f.FileName.split('.')[-1]
            print(exten)
            if exten in ext:
                extcheck=True
    if extcheck==True:
        listt.append(True)
    else:
        listt.append(False)
    if False in listt:
        return False
    else:
        return True

In [ ]:
def send_delayed(message):
    message.SaveAs(r"C:\Users\a-bystrov-ai\messattach.msg")
    NewMsg = outlook.CreateItem(0)
    NewMsg.Attachments.Add(r"C:\Users\a-bystrov-ai\messattach.msg")
    NewMsg.Subject = "[КЛАСТЕР]: "+message.Subject
    NewMsg.To = 'concern@rosenergoatom.ru'
    NewMsg.Send()

In [ ]:
%%time
import glob
import os
import time
from time import gmtime, strftime
import pythoncom           # These 2 lines are here because COM library
import psutil
get_path = r"C:\Users\a-bystrov-ai\Desktop\attach"
list_prove=['uslugi@mos.ru','gr@profeconomy.ru','haf@spepro.ru','pr2@elsib.ru']
obrabotano_all=0
obrabotano_good=0
obrabotano_bad=0
obrabotano_spam=0
obrabotano_spam_false=0
while 1==1:
    try:
        print('new')
        mtrx=pd.read_pickle(r"matrix.pkl")
#         with open('replace_dict.pickle', 'rb') as handle:
#             dicktt2 = pickle.load(handle)
        outlook = client.Dispatch('Outlook.Application')
        namespace = outlook.GetNameSpace("MAPI")
        account=namespace.Folders['info']
        inbox = account.Folders['Входящие']
        messages = inbox.Items
    
        time.sleep(2)
        for i in range(0,100):
            if messages[i].Class==43: #проверка на письмо от человека
                for f in glob.glob(r"C:\Users\a-bystrov-ai\Desktop\attach\*"):
                    os.remove(f)
                if (messages[i].Categories==''):#проверка неоткрытого письма
                    if ktdsp(messages[i],outlook)==False:#проверка КТ ДСП
                        if (check_delay(messages[i]) or ('часть' in messages[i].Subject.lower())):#проверка на кластер
                            print('qwe')
                            messages[i].GetInspector()
                            messages[i].Categories='Delay'
                            messages[i].Save()
                            time.sleep(1)
                            send_delayed(messages[i])
                        elif check_if_pdf()==True:#проверка на пдф
                            fltr = check_matrix(messages[i], outlook)#проверка жесткой матрицей
                            if fltr == 'stop':
                                continue
                            else:
                                predict_pdf(outlook)#пдф ИИ

                        else:
                            predict_nopdf(outlook)#текст ИИ
                    if (messages[i].Categories=='') and (ktdsp(messages[i],outlook)==False):
                        print('Ne perenapravleno strange', messages[i].Subject,strftime("%Y-%m-%d %H:%M:%S", gmtime()))
                        messages[i].GetInspector()
                        messages[i].Categories='Робот(не перенаправлено)'
                        messages[i].Save()                    
                        insert_one(conn,[messages[i].Subject,messages[i].HTMLBody,0,messages[i].Sender.Address,reciever,0,0])
                        time.sleep(1)
                        make_respond_KC(messages[i])


    except Exception as e:
        print(e)
        print(1)
        PROCNAME = "OUTLOOK.EXE"
        for proc in psutil.process_iter():
            if proc.name() == PROCNAME:
                try:
                    p = psutil.Process(proc.pid)
                    p.terminate()
                except:
                    continue
    time.sleep(10)
    

new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new
new


In [ ]:
###### 